## IMPORTAR LAS LIBRERIAS

Actualizar a las que se usen finalmente en tu proyecto.

In [4]:
import numpy as np
import pandas as pd
import cloudpickle

#Automcompletar rápido
%config IPCompleter.greedy=True

from janitor import clean_names
from xgboost import XGBClassifier

from sklearn.preprocessing import StandardScaler
%matplotlib inline
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline

C:\Users\acrespod\Anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
C:\Users\acrespod\Anaconda3\lib\site-packages\xgboost\compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


## CARGAR LOS DATOS

### Ruta del proyecto

In [5]:
ruta_proyecto = 'C:/proyectos/02_CHURN_PREDICTION'

### Nombre del fichero de datos

In [6]:
nombre_fichero_datos =  'customer-churn-prediction-2020.csv'

### Cargar los datos

In [7]:
ruta_completa = ruta_proyecto + '/02_Datos/01_Originales/' + nombre_fichero_datos
df = pd.read_csv(ruta_completa,sep=",")

In [8]:
df

,state,account_length,area_code,international_plan,voice_mail_plan,number_vmail_messages,total_day_minutes,total_day_calls,total_day_charge,total_eve_minutes,total_eve_calls,total_eve_charge,total_night_minutes,total_night_calls,total_night_charge,total_intl_minutes,total_intl_calls,total_intl_charge,number_customer_service_calls,churn
0,OH,107,area_code_415,no,yes,26,161.6,123,27.47,195.5,103,16.62,254.4,103,11.45,13.7,3,3.70,1,no
1,NJ,137,area_code_415,no,no,0,243.4,114,41.38,121.2,110,10.30,162.6,104,7.32,12.2,5,3.29,0,no
2,OH,84,area_code_408,yes,no,0,299.4,71,50.90,61.9,88,5.26,196.9,89,8.86,6.6,7,1.78,2,no
3,OK,75,area_code_415,yes,no,0,166.7,113,28.34,148.3,122,12.61,186.9,121,8.41,10.1,3,2.73,3,no
4,MA,121,area_code_510,no,yes,24,218.2,88,37.09,348.5,108,29.62,212.6,118,9.57,7.5,7,2.03,3,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4245,MT,83,area_code_415,no,no,0,188.3,70,32.01,243.8,88,20.72,213.7,79,9.62,10.3,6,2.78,0,no
4246,WV,73,area_code_408,no,no,0,177.9,89,30.24,131.2,82,11.15,186.2,89,8.38,11.5,6,3.11,3,no
4247,NC,75,area_code_408,no,no,0,170.7,101,29.02,193.1,126,16.41,129.1,104,5.81,6.9,7,1.86,1,no
4248,HI,50,area_code_408,no,yes,40,235.7,127,40.07,223.0,126,18.96,297.5,116,13.39,9.9,5,2.67,2,no


In [36]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
account_length,4250.0,100.236235,39.698401,1.00,73.000000,100.000000,127.000000,243.000000
number_vmail_messages,4250.0,7.631765,13.439882,0.00,0.000000,0.000000,16.000000,52.000000
total_day_minutes,4250.0,180.259600,54.012373,0.00,143.325000,180.450000,216.200000,351.500000
total_day_calls,4250.0,99.907294,19.850817,0.00,87.000000,100.000000,113.000000,165.000000
total_day_charge,4250.0,30.644682,9.182096,0.00,24.365000,30.680000,36.750000,59.760000
total_eve_minutes,4250.0,200.173906,50.249518,0.00,165.925000,200.700000,233.775000,359.300000
total_eve_calls,4250.0,100.176471,19.908591,0.00,87.000000,100.000000,114.000000,170.000000
total_eve_charge,4250.0,17.015012,4.271212,0.00,14.102500,17.060000,19.867500,30.540000
total_night_minutes,4250.0,200.527882,50.353548,0.00,167.225000,200.450000,234.700000,395.000000
total_night_calls,4250.0,99.839529,20.093220,0.00,86.000000,100.000000,113.000000,175.000000


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4250 entries, OH to VT
Data columns (total 19 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   account_length                 4250 non-null   int64  
 1   area_code                      4250 non-null   object 
 2   international_plan             4250 non-null   object 
 3   voice_mail_plan                4250 non-null   object 
 4   number_vmail_messages          4250 non-null   int64  
 5   total_day_minutes              4250 non-null   float64
 6   total_day_calls                4250 non-null   int64  
 7   total_day_charge               4250 non-null   float64
 8   total_eve_minutes              4250 non-null   float64
 9   total_eve_calls                4250 non-null   int64  
 10  total_eve_charge               4250 non-null   float64
 11  total_night_minutes            4250 non-null   float64
 12  total_night_calls              4250 non-null   int64  

### Seleccionar solo las variables finales

#### Cargar la lista de variables finales

In [9]:
v = ['international_plan_no', 'number_customer_service_calls_ss',
       'ratio_state_ss', 'total_day_minutes_ss', 'total_eve_minutes_ss',
       'total_intl_calls_ss', 'total_intl_minutes_ss',
       'total_night_minutes_ss', 'voice_mail_plan_yes']

#### Apuntar (manualmente) la lista de variables finales sin extensiones

In [10]:
variables_finales = ['international_plan', 'number_customer_service_calls',
        'total_day_minutes', 'total_eve_minutes',
       'total_intl_calls', 'total_intl_minutes',
       'total_night_minutes', 'voice_mail_plan','ratio_state']

## ESTRUCTURA DE LOS DATASETS

### Corregir los nombres

In [8]:
df = clean_names(df)

### Eliminar registros

#### Por duplicados

In [9]:
df.drop_duplicates(inplace = True)

#### EDA

#### Para x

Quedarse solo con las de la lista.

In [25]:
x = df[variables_finales].copy()

#### Para y

Especificar la target.

In [26]:
target = 'churn'

Crear el y.

In [27]:
y = df[target].copy()

In [28]:
y

0       no
1       no
2       no
3       no
4       no
        ..
4245    no
4246    no
4247    no
4248    no
4249    no
Name: churn, Length: 4250, dtype: object

## CREAR EL PIPELINE

### Instanciar calidad de datos

#### Crear la función

In [11]:
# winsorizacion
variable = 'number_customer_service_calls'
minimo = 0
maximo = 4
df[variable].clip(minimo,maximo, inplace=True)

# target
df['target'] = np.where(df['churn'] == 'yes', 1, 0)    

medias = df.groupby(['state'])[['target']].apply('mean')
medias_dict = medias.to_dict()['target']
df['ratio_state'] = df['state'].map(medias_dict)


x = df[variables_finales].copy()   
y = df['target'].copy()    


### Instanciar transformación de variables

In [13]:
var_oe = ['international_plan', 'voice_mail_plan']
oe = OneHotEncoder(sparse = False, handle_unknown='ignore')

var_ss = ['total_day_minutes',
'number_customer_service_calls',
'total_intl_calls',
'total_intl_minutes',
'total_eve_minutes',
'total_night_minutes',
'ratio_state']

ss = StandardScaler()


### Crear el pipe del preprocesamiento

#### Crear el column transformer

In [14]:
ct = make_column_transformer(
    (oe, var_oe) ,
    (ss, var_ss),
    remainder='drop')

In [15]:
ct.fit(x)

ColumnTransformer(transformers=[('onehotencoder',
                                 OneHotEncoder(handle_unknown='ignore',
                                               sparse=False),
                                 ['international_plan', 'voice_mail_plan']),
                                ('standardscaler', StandardScaler(),
                                 ['total_day_minutes',
                                  'number_customer_service_calls',
                                  'total_intl_calls', 'total_intl_minutes',
                                  'total_eve_minutes', 'total_night_minutes',
                                  'ratio_state'])])

#### Crear el pipeline del preprocesamiento

In [16]:
pipe_prepro = make_pipeline( ct)

### Instanciar el modelo

#### Instanciar el algoritmo

In [18]:
modelo = XGBClassifier(verbosity=0, 
                       learning_rate= 0.1,
                       max_depth=5,
                       reg_alpha=0,
                       reg_lambda=0.1,
                       scale_pos_weight=5,
                       n_estimators=1000,
                       n_jobs = -1,
                       use_label_encoder=False)


#### Crear el pipe final de entrenamiento

In [19]:
pipe_entrenamiento = make_pipeline(pipe_prepro,modelo)

#### Entrenar el pipe final de ejecución

In [20]:

pipe_ejecucion = pipe_entrenamiento.fit(x,y)

## GUARDAR EL PIPE

### Nombre del pipe final de ejecución

### Guardar el pipe final de ejecución

In [21]:
import pickle

In [23]:
ruta_pipe_ejecucion = ruta_proyecto + '/04_Modelos/' + nombre_pipe_ejecucion

with open(ruta_pipe_ejecucion, mode='wb') as file:
    pickle.dump(pipe_ejecucion, file)

In [24]:
# esto es necesario para la variable real_state
ruta_pipe_lista = ruta_proyecto + '/04_Modelos/' + 'diccionario_state.pickle'

with open(ruta_pipe_lista, mode='wb') as file:
    pickle.dump(medias_dict, file)